In [ ]:
from __future__ import print_function # Use a function definition from future version (say 3.x from 2.7 interpreter)
import pandas as pd
import math
import numpy as np
import os

import cntk as C
import cntk.tests.test_utils
import pickle
import random
from cntk import sequence
from cntk import load_model
from cntk.device import try_set_default_device, gpu,cpu
from scipy.sparse import csr_matrix

cntk.tests.test_utils.set_device_from_pytest_env() # (only needed for our build system)
C.cntk_py.set_fixed_random_seed(1) # fix a random seed for CNTK components
try_set_default_device(gpu(0))


vocab_size = 80000
num_labels = 19
title_size = 52000
body_size  = 210000
input_dim  = vocab_size
label_dim  = num_labels
emb_dim    = 300
hidden_dim = 200

max_length_title = 50
max_length_body  = 200



suffix = "180days_sample"
prefix = "C:\\Users\\t-haohu\\Documents\\Python\\IndustryClassifier\\Data"

data_token_body        = "{}\\middle\\{}_token_body.txt".format(prefix,suffix)
data_train_sample_body = "{}\\middle\\train_{}_body.txt".format(prefix,suffix)
data_test_sample_body  = "{}\\middle\\test_{}_body.txt".format(prefix,suffix)

data_industry_sample = "{}\\ready\\industry_{}.wl".format(prefix,suffix)
data_title_sample    = "{}\\ready\\title_{}.wl".format(prefix,suffix)
data_body_sample     = "{}\\ready\\body_{}.wl".format(prefix,suffix)

input_xt = C.input_variable(shape=(max_length_title),  dtype=np.int)
input_xb = C.input_variable(shape=(max_length_body) ,  dtype=np.int)
input_y  = C.input_variable(shape=(1)               ,  dtype=np.float32)

input_xt_one_hot = C.one_hot(input_xt, num_classes=len(title_dict)   ,  sparse_output=True)
input_xb_one_hot = C.one_hot(input_xb, num_classes=len(body_dict)    ,  sparse_output=True) 
input_y_one_hot = C.one_hot(input_y , num_classes=len(industry_dict) ,  sparse_output=True) 

def load_data_body(input_file,title_dict,body_dict,industry_dict):
    data = open(input_file, encoding = "utf-8").readlines()
    
    data_title = np.zeros((len(data),max_length_title))
    data_body  = np.zeros((len(data),max_length_body))
    data_label = np.zeros((len(data),1))
    
    
    for index,line in enumerate(data):
        row = line.strip("\n").split("\t")
        
        title    =  row[0]
        body     =  row[1]
        industry =  row[2]
        
        for jndex,token in enumerate(title.split(" ")):
            if jndex>=max_length_title:
                break
            data_title[index,jndex]=title_dict.get(token,len(title_dict)-1)
            
        for jndex,token in enumerate(body.split(" ")):
            if jndex>=max_length_body:
                break
            data_body[index,jndex]=body_dict.get(token,len(title_dict)-1)
            
        data_label[index] = industry_dict.get(industry,len(industry_dict))
    return data_title,data_body,data_label

def create_model_cnn_with_body():
    
    h1t= C.layers.Embedding(300,name='embed')(input_xt_one_hot)#init=embedding,
    h1b= C.layers.Embedding(300,name='embed')(input_xb_one_hot)#init=embedding,
    
    #bnb = C.layers.BatchNormalization(name='bn')(h1b)
    #bnt = C.layers.BatchNormalization(name='bn')(h1t)



    h2_1t=C.layers.Convolution((1,emb_dim),num_filters=100,reduction_rank=0,activation=C.relu)(h1t)
    h2_2t=C.layers.Convolution((2,emb_dim),num_filters=100,reduction_rank=0,activation=C.relu)(h1t)
    h2_3t=C.layers.Convolution((3,emb_dim),num_filters=100,reduction_rank=0,activation=C.relu)(h1t)

    h2_1b=C.layers.Convolution((1,emb_dim),num_filters=100,reduction_rank=0,activation=C.relu)(h1b)
    h2_2b=C.layers.Convolution((2,emb_dim),num_filters=100,reduction_rank=0,activation=C.relu)(h1b)
    h2_3b=C.layers.Convolution((3,emb_dim),num_filters=100,reduction_rank=0,activation=C.relu)(h1b)

    h3_2t=C.layers.MaxPooling((max_length_title-1,1),name='pooling_t_1')(h2_2t)
    h3_1t=C.layers.MaxPooling((max_length_title-0,1),name='pooling_t_2')(h2_1t)
    h3_3t=C.layers.MaxPooling((max_length_title-2,1),name='pooling_t_3')(h2_3t)

    h3_2b=C.layers.MaxPooling((max_length_body-1,1),name='pooling_b_1')(h2_2b)
    h3_1b=C.layers.MaxPooling((max_length_body-0,1),name='pooling_b_2')(h2_1b)
    h3_3b=C.layers.MaxPooling((max_length_body-2,1),name='pooling_b_3')(h2_3b)

    h3=C.splice(h3_2t,h3_1t,h3_3t,h3_2b,h3_1b,h3_3b,axis=0)

    h4=C.layers.Dense(hidden_dim, activation=C.relu,name='hidden')(h3)
    drop2 = C.layers.Dropout(keep_prob=0.5)(h4)

    h5=C.layers.Dense(num_labels,name='classify')(drop2)

    return h5

def batch_iter(data,batch_size, num_epochs, shuffle=True):
    # Generates a batch iterator for a dataset.
    data_size = len(data)
    num_batches_per_epoch = int((data_size-1)/batch_size) + 1
    print('data_size: ', data_size, 'batch_size: ', batch_size, 'num_batches_per_epoch: ', num_batches_per_epoch)
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            random.shuffle(data)
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield data[start_index:end_index]
            
def transfer(_tuple):
    _list = list(_tuple)
    return [item.tolist() for item in _list]

title_dict =     { x:i for i,x in enumerate([x.strip("\n") for x in open(data_title_sample).readlines()])}
body_dict  =     { x:i for i,x in enumerate([x.strip("\n") for x in open(data_body_sample ).readlines()])}
industry_dict =  { x:i for i,x in enumerate([x.strip("\n") for x in open(data_industry_sample).readlines()])}


test_data_title, test_data_body, test_data_label  = load_data_body(data_test_sample_body ,title_dict,body_dict,industry_dict)



def train(num_epochs,learning_rate,batch_size):
    
    
    model = create_model_cnn_with_body()
    print(C.logging.get_node_outputs(model))
    scores = model(input_xt,input_xb)

    loss =C.reduce_mean(C.losses.cross_entropy_with_softmax(scores, input_y_one_hot))
    
    # Training
    lr_schedule = C.learning_parameter_schedule(learning_rate)
    learner = C.adam(scores.parameters, lr=lr_schedule, momentum=0.9,l2_regularization_weight=0.0001)
    progress_printer = C.logging.ProgressPrinter(tag='Training', num_epochs=num_epochs)
    
    trainer = C.Trainer(scores, (loss), [learner], progress_printer)
    
    train_data_title,train_data_body,train_data_label = load_data_body(data_train_sample_body,title_dict,body_dict,industry_dict)
    batches = batch_iter(list(zip(train_data_title,train_data_body,train_data_label)), batch_size, num_epochs)

    # training loop
    count = 0
    for batch in batches:
        count += 1
        batch_data_title,batch_data_body,batch_data_label = zip(*batch)
        #batch_data_title,batch_data_body,batch_data_label = transfer(batch_data_title),transfer(batch_data_body),trainsfer(batch_data_label)
        trainer.train_minibatch({input_xb: np.array(batch_data_body),input_xt: np.array(batch_data_title), input_y: np.array(batch_data_label)})

        

    # save model
    scores.save('./model/IndustryClassifyCNN_body.dnn')


In [ ]:
train(num_epochs=10,learning_rate=1e-4,batch_size = 50)

In [ ]:
x = [1,2,3]
y = [4,5,6]
zipped = zip(x,y)
unzipped_x, unzipped_y = zip(*zipped)
print(list(zipped),unzipped_x)
